In [66]:
import pandas as pd
import mariadb
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from supervised.automl import AutoML
from tqdm import tqdm
import numpy as np
import pandas as pd
import numpy as np
import bisect
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
from supervised.automl import AutoML


conn  = mariadb.connect(
    user='team3', 
    password='dbdb', 
    database='team3db', 
    host='localhost',
    port=3306
)

# 위 커넥션 정보와 동일하게 입력
engine = create_engine("mysql://{user}:{pw}@{host}/{db}".format(user='team3', pw='dbdb',host='localhost', db='team3db'))

In [67]:
train = pd.read_sql_query('SELECT * FROM train3',engine,index_col='index')
train

,ARI_CO,SHIP_TYPE_CATEGORY,DIST,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,FLAG,U_WIND,AIR_TEMPERATURE,PORT_SIZE,CI_HOUR,year,weekday
index,,,,,,,,,,,,,,,,,
3,JP,Container,0.000000,20.0,18.0,6910,10.0,10.0,5400,120.0,Bahamas,-3.18,6.7,0.000356,0.000000,2020,4
4,SG,Container,27.037650,50.0,10.0,116000,20.0,10.0,96600,300.0,Liberia,-0.33,25.6,0.002615,253.554444,2020,6
5,AU,Bulk,49.953585,40.0,7.0,183000,20.0,20.0,94100,290.0,Japan,6.10,28.1,0.000103,68.391389,2021,4
7,TW,Cargo,0.000000,30.0,18.0,46600,20.0,10.0,29800,200.0,"Hong Kong, China",2.82,28.6,0.000990,0.000000,2022,4
10,CN,Bulk,18.022495,30.0,11.0,35000,20.0,10.0,22500,180.0,Montenegro,-2.16,25.6,0.000520,2.721944,2021,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391928,AU,Bulk,55.601755,50.0,8.0,210000,20.0,20.0,107000,300.0,Liberia,6.97,24.9,0.000103,55.701667,2021,1
391929,SG,Container,7.304235,40.0,9.0,51700,20.0,10.0,41700,230.0,Singapore,-0.98,26.1,0.002615,23.914444,2019,1
391933,CN,Bulk,9.647703,30.0,26.0,69000,20.0,10.0,35900,220.0,Panama,3.64,21.4,0.000939,1095.597222,2020,0


In [68]:
train.reset_index(inplace=True,drop=True)
train

,ARI_CO,SHIP_TYPE_CATEGORY,DIST,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,FLAG,U_WIND,AIR_TEMPERATURE,PORT_SIZE,CI_HOUR,year,weekday
0,JP,Container,0.000000,20.0,18.0,6910,10.0,10.0,5400,120.0,Bahamas,-3.18,6.7,0.000356,0.000000,2020,4
1,SG,Container,27.037650,50.0,10.0,116000,20.0,10.0,96600,300.0,Liberia,-0.33,25.6,0.002615,253.554444,2020,6
2,AU,Bulk,49.953585,40.0,7.0,183000,20.0,20.0,94100,290.0,Japan,6.10,28.1,0.000103,68.391389,2021,4
3,TW,Cargo,0.000000,30.0,18.0,46600,20.0,10.0,29800,200.0,"Hong Kong, China",2.82,28.6,0.000990,0.000000,2022,4
4,CN,Bulk,18.022495,30.0,11.0,35000,20.0,10.0,22500,180.0,Montenegro,-2.16,25.6,0.000520,2.721944,2021,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217103,AU,Bulk,55.601755,50.0,8.0,210000,20.0,20.0,107000,300.0,Liberia,6.97,24.9,0.000103,55.701667,2021,1
217104,SG,Container,7.304235,40.0,9.0,51700,20.0,10.0,41700,230.0,Singapore,-0.98,26.1,0.002615,23.914444,2019,1
217105,CN,Bulk,9.647703,30.0,26.0,69000,20.0,10.0,35900,220.0,Panama,3.64,21.4,0.000939,1095.597222,2020,0
217106,SG,Bulk,5.884603,10.0,12.0,3160,10.0,10.0,1600,80.0,Vietnam,-0.66,27.3,0.002615,144.061389,2019,2


In [69]:
test = pd.read_sql_query('SELECT * FROM test',engine, index_col='index')
test.reset_index(drop=True,inplace=True)
test

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,GT,LENGTH,SHIPMANAGER,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE
0,TEST_000000,SG,GIW5,Container,1.826589,2020-06-18 11:58,K322654,50.0,18,117000,...,109000,340.0,KQMD74,Panama,0.37,1.63,27.1,1.587063,19,0.002615
1,TEST_000001,CN,WEY7,Cargo,25.399386,2021-05-26 22:20,E376681,10.0,13,3810,...,2560,80.0,LBYS27,Belize,-2.79,-2.33,14.2,2.663972,6,0.001028
2,TEST_000002,CN,NGG6,Container,111.079467,2019-12-16 0:09,F811111,20.0,26,10900,...,8440,140.0,EKFV15,Singapore,0.04,-4.91,9.3,3.255315,8,0.001743
3,TEST_000003,CA,FFM2,Bulk,9.175258,2015-11-16 5:30,A737561,30.0,9,55800,...,31500,190.0,MASW32,Panama,NaN,NaN,NaN,NaN,2,0.000182
4,TEST_000004,JP,QYY1,Container,0.000000,2018-10-24 1:11,A827175,30.0,19,39800,...,33000,220.0,SIEL54,Liberia,NaN,NaN,NaN,NaN,10,0.000552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220486,TEST_220486,BR,TMW2,Bulk,21.712733,2017-04-04 15:53,E731427,60.0,15,298000,...,151000,330.0,UWHE18,Japan,NaN,NaN,NaN,NaN,13,0.000080
220487,TEST_220487,CA,GRQ5,Bulk,4.870490,2016-08-22 16:22,Q434522,30.0,9,80500,...,42700,220.0,BBXV46,Liberia,NaN,NaN,NaN,NaN,13,0.000039
220488,TEST_220488,JP,VYJ1,Tanker,17.068286,2022-07-10 14:53,U148516,10.0,14,1200,...,500,60.0,DYUA26,Japan,-2.87,1.22,27.4,2.405268,23,0.000264
220489,TEST_220489,CN,QQW1,Cargo,0.000000,2020-12-28 14:38,C235264,10.0,27,3420,...,2000,80.0,VKVS56,Belize,-2.65,-0.64,8.2,2.199039,22,0.000595


In [71]:
test.isnull().sum()

SAMPLE_ID                 0
ARI_CO                    0
ARI_PO                    0
SHIP_TYPE_CATEGORY        0
DIST                      0
ATA                       0
ID                        0
BREADTH                   0
BUILT                     0
DEADWEIGHT                0
DEPTH                     0
DRAUGHT                   0
GT                        0
LENGTH                    0
SHIPMANAGER               0
FLAG                      0
U_WIND                91720
V_WIND                91720
AIR_TEMPERATURE       92241
BN                    91720
ATA_LT                    0
PORT_SIZE                 0
dtype: int64

In [72]:
test.columns

Index(['SAMPLE_ID', 'ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'DIST', 'ATA',
       'ID', 'BREADTH', 'BUILT', 'DEADWEIGHT', 'DEPTH', 'DRAUGHT', 'GT',
       'LENGTH', 'SHIPMANAGER', 'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE',
       'BN', 'ATA_LT', 'PORT_SIZE'],
      dtype='object')

In [73]:
test_port = test.groupby('ARI_PO')['U_WIND'].mean().to_dict()

for port, mean_temp in test_port.items():
    test.loc[test['ARI_PO'] == port, 'U_WIND'] = test.loc[test['ARI_PO'] == port, 'U_WIND'].fillna(mean_temp)
    

test['U_WIND'].isnull().sum()


0

In [74]:
test_port = test.groupby('ARI_PO')['AIR_TEMPERATURE'].mean().to_dict()

for port, mean_temp in test_port.items():
    test.loc[test['ARI_PO'] == port, 'AIR_TEMPERATURE'] = test.loc[test['ARI_PO'] == port, 'AIR_TEMPERATURE'].fillna(mean_temp)
    

test['AIR_TEMPERATURE'].isnull().sum()


0

In [75]:
test_port = test.groupby('ARI_PO')['BN'].mean().to_dict()

for port, mean_temp in test_port.items():
    test.loc[test['ARI_PO'] == port, 'BN'] = test.loc[test['ARI_PO'] == port, 'BN'].fillna(mean_temp)
    

test['BN'].isnull().sum()

0

In [76]:
test.drop(columns=['SAMPLE_ID','ID','ARI_PO','V_WIND'],inplace=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_31112\2089142977.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [77]:
test.shape

(220486, 18)

In [78]:
test.isnull().sum()

ARI_CO                0
SHIP_TYPE_CATEGORY    0
DIST                  0
ATA                   0
BREADTH               0
BUILT                 0
DEADWEIGHT            0
DEPTH                 0
DRAUGHT               0
GT                    0
LENGTH                0
SHIPMANAGER           0
FLAG                  0
U_WIND                0
AIR_TEMPERATURE       0
BN                    0
ATA_LT                0
PORT_SIZE             0
dtype: int64

In [79]:
categorical_features = ['ARI_CO', 'SHIP_TYPE_CATEGORY',  'FLAG']
encoders = {}

for feature in tqdm(categorical_features, desc="Encoding features"):
    le = LabelEncoder()
    train[feature] = le.fit_transform(train[feature].astype(str))
    le_classes_set = set(le.classes_)
    test[feature] = test[feature].map(lambda s: '-1' if s not in le_classes_set else s)
    le_classes = le.classes_.tolist()
    bisect.insort_left(le_classes, '-1')
    le.classes_ = np.array(le_classes)
    test[feature] = le.transform(test[feature].astype(str))
    encoders[feature] = le

Encoding features:   0%|          | 0/3 [00:00<?, ?it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_31112\3896781814.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\AppData\Local\Temp\ipykernel_31112\3896781814.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Encoding features:  33%|███▎      | 1/3 [00:00<00:00,  6.07it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_31112\3896781814.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [80]:
test['ATA'] = pd.to_datetime(test['ATA'])
test['year'] = test['ATA'].dt.year
test['weekday'] = test['ATA'].dt.weekday
test.drop(columns=['ATA','ATA_LT','BN','SHIPMANAGER'], inplace=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_31112\2263708381.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\AppData\Local\Temp\ipykernel_31112\2263708381.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\AppData\Local\Temp\ipykernel_31112\2263708381.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [81]:
X_train = train.drop(columns='CI_HOUR')
Y_train = train['CI_HOUR']

In [82]:
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(X_train)
test_scaled = scaler.transform(test)

In [83]:
X_train = pd.DataFrame(train_scaled, columns = X_train.columns)
test = pd.DataFrame(test_scaled, columns = test.columns)

In [84]:
automl = AutoML(
    algorithms=["CatBoost", "Xgboost", "LightGBM", "Random Forest",'Neural Network'],
    mode="Compete",
    ml_task="regression",
    eval_metric='mae',
    random_state=42,
    validation_strategy={
    "validation_type": "kfold",
    "k_folds": 5,
    "shuffle": True,
    "random_seed": 42
},
    total_time_limit=None,
    model_time_limit=None
)

In [85]:
automl = AutoML(
    algorithms=["CatBoost", "Xgboost", "LightGBM", "Random Forest"],
    mode="Compete",
    ml_task="regression",
    eval_metric='r2',
    random_state=42,
    total_time_limit=None,
    model_time_limit=None
)

In [86]:
automl.fit(X_train, Y_train)
pred = automl.predict(test)

AutoML directory: AutoML_7
The task is regression with evaluation metric r2
AutoML will use algorithms: ['CatBoost', 'Xgboost', 'LightGBM', 'Random Forest']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 4 models
1_Default_LightGBM r2 0.288291 trained in 58.83 seconds
2_Default_Xgboost r2 0.279021 trained in 56.37 seconds
3_Default_CatBoost r2 0.277897 trained in 69.67 seconds
4_Default_RandomForest r2 0.13479 trained in 67.16 seconds
* Step not_so_random will try to check up to 36 models
14_LightGBM r2 0.290345 trained in 53.4 seconds
5_Xgboost r2 0.284735 trained in 44.72 seconds
23_CatBoost r2 0.285424 tr

In [92]:
submit = pd.read_csv('./data/sample_submission.csv')
submit['CI_HOUR'] = pred
submit.to_csv('./csv/submit.csv', index=False)